In [1]:
import warnings
warnings.filterwarnings(action='ignore')

from pyspark.sql import SparkSession

spark = SparkSession.builder \
           .appName('spark_sql_study') \
           .config("spark.jars", "/home/yms06034/spark/spark3/mysql-connector-java-8.0.29.jar") \
           .getOrCreate()

23/03/24 11:25:03 WARN Utils: Your hostname, DESKTOP-IV4R8HD resolves to a loopback address: 127.0.1.1; using 192.168.62.80 instead (on interface eth0)
23/03/24 11:25:03 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/03/24 11:25:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
spark

In [3]:
customers = spark.read.format("jdbc").option("url", "jdbc:mysql://localhost:3306/spark_db") \
    .option("driver", "com.mysql.jdbc.Driver").option("dbtable", "customers") \
    .option("user", "root").option("password", "").load()

geolocation = spark.read.format("jdbc").option("url", "jdbc:mysql://localhost:3306/spark_db") \
    .option("driver", "com.mysql.jdbc.Driver").option("dbtable", "geolocation") \
    .option("user", "root").option("password", "").load()

order_items = spark.read.format("jdbc").option("url", "jdbc:mysql://localhost:3306/spark_db") \
    .option("driver", "com.mysql.jdbc.Driver").option("dbtable", "order_items") \
    .option("user", "root").option("password", "").load()

order_payments = spark.read.format("jdbc").option("url", "jdbc:mysql://localhost:3306/spark_db") \
    .option("driver", "com.mysql.jdbc.Driver").option("dbtable", "order_payments") \
    .option("user", "root").option("password", "").load()

order_reviews = spark.read.format("jdbc").option("url", "jdbc:mysql://localhost:3306/spark_db") \
    .option("driver", "com.mysql.jdbc.Driver").option("dbtable", "order_reviews") \
    .option("user", "root").option("password", "").load()

orders = spark.read.format("jdbc").option("url", "jdbc:mysql://localhost:3306/spark_db") \
    .option("driver", "com.mysql.jdbc.Driver").option("dbtable", "orders") \
    .option("user", "root").option("password", "").load()

product_category_name_translation = spark.read.format("jdbc").option("url", "jdbc:mysql://localhost:3306/spark_db") \
    .option("driver", "com.mysql.jdbc.Driver").option("dbtable", "product_category_name_translation") \
    .option("user", "root").option("password", "").load()

products = spark.read.format("jdbc").option("url", "jdbc:mysql://localhost:3306/spark_db") \
    .option("driver", "com.mysql.jdbc.Driver").option("dbtable", "products") \
    .option("user", "root").option("password", "").load()

sellers = spark.read.format("jdbc").option("url", "jdbc:mysql://localhost:3306/spark_db") \
    .option("driver", "com.mysql.jdbc.Driver").option("dbtable", "sellers") \
    .option("user", "root").option("password", "").load()

Loading class `com.mysql.jdbc.Driver'. This is deprecated. The new driver class is `com.mysql.cj.jdbc.Driver'. The driver is automatically registered via the SPI and manual loading of the driver class is generally unnecessary.


In [4]:
print('[customers]')
customers.printSchema()
print('--' * 50)

print('[geolocation]')
geolocation.printSchema()
print('--' * 50)

print('[order_items]')
order_items.printSchema()
print('--' * 50)

print('[order_payments]')
order_payments.printSchema()
print('--' * 50)

print('[order_reviews]')
order_reviews.printSchema()
print('--' * 50)

print('[orders]')
orders.printSchema()
print('--' * 50)

print('[product_category_name_translation]')
product_category_name_translation.printSchema()
print('--' * 50)

print('[products]')
products.printSchema()
print('--' * 50)

print('[sellers]')
sellers.printSchema()
print('--' * 50)

[customers]
root
 |-- index: long (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- customer_unique_id: string (nullable = true)
 |-- customer_zip_code_prefix: long (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- customer_state: string (nullable = true)

----------------------------------------------------------------------------------------------------
[geolocation]
root
 |-- index: long (nullable = true)
 |-- geolocation_zip_code_prefix: long (nullable = true)
 |-- geolocation_lat: double (nullable = true)
 |-- geolocation_lng: double (nullable = true)
 |-- geolocation_city: string (nullable = true)
 |-- geolocation_state: string (nullable = true)

----------------------------------------------------------------------------------------------------
[order_items]
root
 |-- index: long (nullable = true)
 |-- order_id: string (nullable = true)
 |-- order_item_id: long (nullable = true)
 |-- product_id: string (nullable = true)
 |-- seller_id: string (n

In [5]:
customers.createOrReplaceTempView("customers")
geolocation.createOrReplaceTempView("geolocation")
order_items.createOrReplaceTempView("order_items")
order_payments.createOrReplaceTempView("order_payments")
order_reviews.createOrReplaceTempView("order_reviews")
orders.createOrReplaceTempView("orders")
product_category_name_translation.createOrReplaceTempView("product_category_name_translation")
products.createOrReplaceTempView("products")
sellers.createOrReplaceTempView("sellers")

### Orders

In [6]:
query = """
SELECT 
    *
FROM 
    orders;
"""

In [7]:
spark.sql(query).show()

+-----+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|index|            order_id|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+-----+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|    0|e481f51cbdc54678b...|9ef432eb625129730...|   delivered|     2017-10-02 10:56:33|2017-10-02 11:07:15|         2017-10-04 19:55:00|          2017-10-10 21:25:13|          2017-10-18 00:00:00|
|    1|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|   delivered|     2018-07-24 20:41:37|2018-07-26 03:24:27|         2018-07-26 14:31:00|          2018-08-07 15:27:45|          2018-08-13 00:00:00|
|    2|47770eb9

In [8]:
query = """
SELECT 
    DISTINCT(order_status)
FROM 
    orders;
"""

In [9]:
spark.sql(query).show()

+------------+
|order_status|
+------------+
|     shipped|
|    canceled|
|    approved|
|    invoiced|
|     created|
|   delivered|
| unavailable|
|  processing|
+------------+



In [10]:
query = """
SELECT 
    customer_id, COUNT(order_id) as order_count
FROM 
    orders
WHERE 
    order_status == 'delivered'
GROUP BY 
    customer_id
ORDER BY 2 DESC;
"""

In [11]:
spark.sql(query).show()

+--------------------+-----------+
|         customer_id|order_count|
+--------------------+-----------+
|ef15b3240b2083e04...|          1|
|70ab5e994ea383619...|          1|
|0df487c45aada9cfd...|          1|
|b8c5e65ce798bc0c8...|          1|
|743dd95aefb65359a...|          1|
|df0aa5b8586495e0d...|          1|
|9f4578f809480da7c...|          1|
|116a19a0ae346ac66...|          1|
|0c61d5a89f3c2614a...|          1|
|039c5d6f330c73420...|          1|
|8bdba2ea966d946cc...|          1|
|7c029959cc691fbe5...|          1|
|874dfb94ae7ceab04...|          1|
|5f46e687a5c82cb1e...|          1|
|c69e9b9095b1491d5...|          1|
|72c990123f7cab26c...|          1|
|36b9a428f314355ab...|          1|
|aee9d44d59c6cb00a...|          1|
|cdfb43f4e84e1508f...|          1|
|bc94fe20172cefc3e...|          1|
+--------------------+-----------+
only showing top 20 rows



In [17]:
query = """
SELECT 
    customer_id, COUNT(order_id) as order_count
FROM 
    orders
WHERE 
    order_status == 'delivered'
GROUP BY 
    customer_id
ORDER BY 2 DESC;
"""

In [18]:
spark.sql(query).show()

+--------------------+-----------+
|         customer_id|order_count|
+--------------------+-----------+
|f54a9f0e6b351c431...|          1|
|2a1dfb647f32f4390...|          1|
|4f28355e5c17a4a42...|          1|
|4632eb5a8f175f6fe...|          1|
|843ff05b30ce4f75b...|          1|
|a4156bb8aff5d6722...|          1|
|1099d033c74a027a7...|          1|
|7a3bd3b37285f0ab2...|          1|
|5fff39f1b59dc4d2f...|          1|
|18f6ca10777417c93...|          1|
|f1e46939e6408b3e6...|          1|
|a8695124db570d100...|          1|
|545b9a267af9ba134...|          1|
|90d7075599361b694...|          1|
|a340ce6c3570e68d4...|          1|
|ce0681e1c3f70e145...|          1|
|ee905ec97794ec6e9...|          1|
|b74ca180d63f9ae04...|          1|
|0de46efc7d10114ac...|          1|
|8bdae6b4ff9bc7f4c...|          1|
+--------------------+-----------+
only showing top 20 rows



In [27]:
query = """
SELECT 
    order_status, COUNT(order_id)
FROM 
    orders
GROUP BY 
    order_status
ORDER BY
    2 DESC;
"""

In [28]:
spark.sql(query).show()

+------------+---------------+
|order_status|count(order_id)|
+------------+---------------+
|   delivered|          96478|
|     shipped|           1107|
|    canceled|            625|
| unavailable|            609|
|    invoiced|            314|
|  processing|            301|
|     created|              5|
|    approved|              2|
+------------+---------------+



In [12]:
query = """
SELECT 
    order_id, order_status, order_purchase_timestamp
FROM 
    orders
ORDER BY 
    order_purchase_timestamp DESC;
"""

In [13]:
spark.sql(query).show()

+--------------------+------------+------------------------+
|            order_id|order_status|order_purchase_timestamp|
+--------------------+------------+------------------------+
|10a045cdf6a5650c2...|    canceled|     2018-10-17 17:30:18|
|b059ee4de278302d5...|    canceled|     2018-10-16 20:16:02|
|a2ac6dad85cf8af5b...|    canceled|     2018-10-03 18:55:29|
|616fa7d4871b87832...|    canceled|     2018-10-01 15:30:09|
|392ed9afd714e3c74...|    canceled|     2018-09-29 09:13:03|
|869997fbe01f39d18...|    canceled|     2018-09-26 08:40:15|
|5aac76cf7b07dd06f...|    canceled|     2018-09-25 11:59:18|
|ed3efbd3a87bea76c...|    canceled|     2018-09-20 13:54:16|
|bd35b677fd239386e...|    canceled|     2018-09-17 17:21:16|
|ea844c92cf978ea23...|    canceled|     2018-09-13 09:56:12|
|1ad22e16129698e1a...|    canceled|     2018-09-12 15:32:16|
|87ae60ef8b08ae0e5...|    canceled|     2018-09-11 16:45:54|
|21a00b08cbeb5716b...|    canceled|     2018-09-10 17:51:25|
|a89abace0dcc01eeb...|  

In [29]:
query = """
SELECT 
    order_id, order_status, order_purchase_timestamp
FROM 
    orders
ORDER BY 
    order_purchase_timestamp ASC;
"""

spark.sql(query).show()

+--------------------+------------+------------------------+
|            order_id|order_status|order_purchase_timestamp|
+--------------------+------------+------------------------+
|2e7a8482f6fb09756...|     shipped|     2016-09-04 21:15:19|
|e5fa5a7210941f7d5...|    canceled|     2016-09-05 00:15:34|
|809a282bbd5dbcabb...|    canceled|     2016-09-13 15:24:19|
|bfbd0f9bdef843021...|   delivered|     2016-09-15 12:16:38|
|71303d7e93b399f5b...|    canceled|     2016-10-02 22:07:52|
|3b697a20d9e427646...|   delivered|     2016-10-03 09:44:50|
|be5bc2f0da14d8071...|   delivered|     2016-10-03 16:56:50|
|65d1e226dfaeb8cdc...|    canceled|     2016-10-03 21:01:41|
|a41c8759fbe7aab36...|   delivered|     2016-10-03 21:13:36|
|d207cc272675637bf...|   delivered|     2016-10-03 22:06:03|
|cd3b8574c82b42fc8...|   delivered|     2016-10-03 22:31:31|
|ae8a60e4b03c5a4ba...|   delivered|     2016-10-03 22:44:10|
|ef1b29b591d31d57c...|   delivered|     2016-10-03 22:51:30|
|0a0837a5eee9e7a9c...|  

In [30]:
query = """
SELECT 
    order_id, customer_id, order_status,
    ROUND(TO_DAYS(order_approved_at) - TO_DAYS(order_purchase_timestamp)+1721060) AS buy_to_approved,
    ROUND(TO_DAYS(order_delivered_carrier_date)+1721060 - TO_DAYS(order_approved_at)+1721060) AS aproved_to_carrier,
    ROUND(TO_DAYS(order_estimated_delivery_date)+1721060 - TO_DAYS(order_delivered_carrier_date)+1721060) AS carrier_to_delivered,
    ROUND(TO_DAYS(order_delivered_customer_date)+1721060 - TO_DAYS(order_delivered_carrier_date)+1721060) AS carrier_to_customer,
    ( ROUND(TO_DAYS(order_delivered_customer_date)+1721060 - TO_DAYS(order_delivered_carrier_date)+1721060) - ROUND(TO_DAYS(order_estimated_delivery_date)+1721060 - TO_DAYS(order_delivered_carrier_date)+1721060)) AS estimated
FROM 
    orders
;
"""

In [32]:
# spark.sql(query).show()

In [44]:
query = """
SELECT
    order_purchase_date, COUNT(*) as purchased_volume
FROM
    (SELECT split(order_purchase_timestamp, ' ')[0] as order_purchase_date FROM orders)
GROUP BY 
    order_purchase_date
ORDER BY
    2 DESC;
"""

spark.sql(query).show()

+-------------------+----------------+
|order_purchase_date|purchased_volume|
+-------------------+----------------+
|         2017-11-24|            1176|
|         2017-11-25|             499|
|         2017-11-27|             403|
|         2017-11-26|             391|
|         2017-11-28|             380|
|         2018-05-07|             372|
|         2018-08-06|             372|
|         2018-08-07|             370|
|         2018-05-14|             364|
|         2018-05-16|             357|
|         2018-05-15|             352|
|         2018-05-09|             344|
|         2017-12-04|             337|
|         2018-05-08|             331|
|         2017-11-29|             323|
|         2018-07-31|             322|
|         2018-08-16|             320|
|         2018-08-14|             316|
|         2018-08-08|             316|
|         2018-01-22|             314|
+-------------------+----------------+
only showing top 20 rows



In [47]:
query = """
SELECT 
    payment_sequential, COUNT(order_id), AVG(payment_value)
FROM 
    order_payments
GROUP BY 
    payment_sequential
ORDER BY 
    2 DESC, 1;
"""

spark.sql(query).show()

+------------------+---------------+------------------+
|payment_sequential|count(order_id)|avg(payment_value)|
+------------------+---------------+------------------+
|                 1|          99360|158.33677365135628|
|                 2|           3039|  79.1018492925306|
|                 3|            581|30.417710843373516|
|                 4|            278|23.265431654676256|
|                 5|            170| 23.89588235294118|
|                 6|            118| 24.06161016949153|
|                 7|             82|20.672439024390247|
|                 8|             54|13.437407407407408|
|                 9|             43|13.521627906976748|
|                10|             34|15.617352941176474|
|                11|             29|13.351379310344829|
|                12|             21| 16.07761904761905|
|                13|             13| 8.773846153846154|
|                14|             10|            13.481|
|                15|              8|12.434999999

In [14]:
query = """
SELECT 
    payment_type, COUNT(order_id), AVG(payment_value)
FROM 
    order_payments
GROUP BY 
    payment_type
ORDER BY 2 DESC;
"""

In [15]:
spark.sql(query).show()

+------------+---------------+------------------+
|payment_type|count(order_id)|avg(payment_value)|
+------------+---------------+------------------+
| credit_card|          76795|163.31902063935578|
|      boleto|          19784| 145.0344354023441|
|     voucher|           5775| 65.70335411255424|
|  debit_card|           1529|142.57017004578162|
| not_defined|              3|               0.0|
+------------+---------------+------------------+



In [48]:
query = """
SELECT 
    payment_installments, COUNT(order_id), AVG(payment_value)
FROM 
    order_payments
GROUP BY 
    payment_installments
ORDER BY 
    1 ASC, 2 DESC;
"""

spark.sql(query).show()

+--------------------+---------------+------------------+
|payment_installments|count(order_id)|avg(payment_value)|
+--------------------+---------------+------------------+
|                   0|              2|            94.315|
|                   1|          52546|112.42022913256561|
|                   2|          12413|127.22815032627128|
|                   3|          10461| 142.5393174648697|
|                   4|           7098|163.97683995491641|
|                   5|           5239|183.46522237068183|
|                   6|           3920| 209.8499515306125|
|                   7|           1626| 187.6736715867158|
|                   8|           4268|307.73742736644886|
|                   9|            644|203.44086956521718|
|                  10|           5328| 415.0858370870872|
|                  11|             23| 124.9321739130435|
|                  12|            133| 321.6784962406015|
|                  13|             16|          150.4625|
|             

In [56]:
query = """
SELECT 
  order_id, 
  MAX(payment_sequential) as payment_sequential, 
  MAX(payment_type) as payment_type, 
  MAX(payment_installments) as payment_installments, 
  AVG(payment_value) as avg_payment_value
FROM 
    order_payments
GROUP BY 
    order_id;
-- ORDER BY 4 DESC
"""

spark.sql(query).show()

+--------------------+------------------+------------+--------------------+-----------------+
|            order_id|payment_sequential|payment_type|payment_installments|avg_payment_value|
+--------------------+------------------+------------+--------------------+-----------------+
|00018f77f2f0320c5...|                 1| credit_card|                   3|           259.83|
|00042b26cf59d7ce6...|                 1| credit_card|                   3|           218.04|
|00054e8431b9d7675...|                 1| credit_card|                   1|            31.75|
|0005a1a1728c9d785...|                 1| credit_card|                   3|            157.6|
|0005f50442cb953dc...|                 1| credit_card|                   1|            65.39|
|00063b381e2406b52...|                 1| credit_card|                   5|            57.98|
|0006ec9db01a64e59...|                 1| credit_card|                   4|            97.32|
|000aed2e25dbad2f9...|                 1| credit_card|      

In [58]:
query = """
SELECT 
    *
FROM 
    order_reviews;
"""

spark.sql(query).show()

+--------------------+--------------------+------------+--------------------+-----------------------+
|           review_id|            order_id|review_score|review_creation_date|review_answer_timestamp|
+--------------------+--------------------+------------+--------------------+-----------------------+
|7bc2406110b926393...|73fc7af87114b3971...|           4|     2018-01-18 0:00|       2018-01-18 21:46|
|80e641a11e56f04c1...|a548910a1c6147796...|           5|     2018-03-10 0:00|        2018-03-11 3:05|
|228ce5500dc1d8e02...|f9e4b658b201a9f2e...|           5|     2018-02-17 0:00|       2018-02-18 14:36|
|e64fb393e7b32834b...|658677c97b385a9be...|           5|     2017-04-21 0:00|       2017-04-21 22:02|
|f7c4243c7fe1938f1...|8e6bfb81e283fa7e4...|           5|     2018-03-01 0:00|       2018-03-02 10:26|
|15197aa66ff4d0650...|b18dcdf73be663668...|           1|     2018-04-13 0:00|        2018-04-16 0:39|
|07f9bee5d1b850860...|e48aa0d2dcec3a2e8...|           5|     2017-07-16 0:00|     

In [69]:
query = """
SELECT 
    order_id, COUNT(review_id) as reviewId_count, MAX(review_creation_date) as review_creation_date, AVG(review_score) as review_score
FROM 
    order_reviews
GROUP BY 
    order_id
HAVING 
    COUNT(review_id) > 1
ORDER BY
    3 DESC;
"""

spark.sql(query).show()

+--------------------+--------------+--------------------+------------+
|            order_id|reviewId_count|review_creation_date|review_score|
+--------------------+--------------+--------------------+------------+
|f3ee55bf50ef2f231...|             2|     2018-08-29 0:00|         4.5|
|84aa61a900410cfe2...|             2|     2018-08-22 0:00|         5.0|
|dab8a6c6bd6ec448d...|             2|     2018-08-22 0:00|         3.5|
|80fe5b2fe4869330f...|             2|     2018-08-11 0:00|         5.0|
|3e93213bb8fdda911...|             2|     2018-08-08 0:00|         5.0|
|0b5588f04551d90ee...|             2|     2018-08-04 0:00|         3.0|
|77ef3216467887c78...|             2|     2018-08-01 0:00|         5.0|
|b76e1e63723d846f3...|             2|     2018-08-01 0:00|         5.0|
|230429e6f253f9cd6...|             2|     2018-08-01 0:00|         5.0|
|715b8576b74d53796...|             2|     2018-08-01 0:00|         5.0|
|a1b8aea9fd9e26a86...|             2|     2018-07-31 0:00|      

In [77]:
query = """
SELECT 
    *
FROM 
    order_reviews
WHERE 
    order_id = '03c939fd7fd3b38f8485a0f95798f1f6';
-- ORDER BY review_answer_timestamp ASC;
"""

spark.sql(query).show()

+--------------------+--------------------+------------+--------------------+-----------------------+
|           review_id|            order_id|review_score|review_creation_date|review_answer_timestamp|
+--------------------+--------------------+------------+--------------------+-----------------------+
|b04ed893318da5b86...|03c939fd7fd3b38f8...|           3|     2018-03-20 0:00|        2018-03-21 2:28|
|f4bb9d6dd4fb6dcc2...|03c939fd7fd3b38f8...|           4|     2018-03-29 0:00|        2018-03-30 0:29|
|405eb2ea45e1dbe26...|03c939fd7fd3b38f8...|           3|     2018-03-06 0:00|       2018-03-06 19:50|
+--------------------+--------------------+------------+--------------------+-----------------------+



In [86]:
query = """
SELECT 
    review_creation_date, COUNT(*) as negative_count
FROM 
    order_reviews
WHERE 
    review_score = 1
GROUP BY
    review_creation_date
ORDER BY 
    2 DESC;
"""

spark.sql(query).show()

+--------------------+--------------+
|review_creation_date|negative_count|
+--------------------+--------------+
|     2018-03-24 0:00|           103|
|     2018-03-23 0:00|           101|
|     2018-03-21 0:00|            95|
|     2017-12-20 0:00|            93|
|     2017-12-19 0:00|            90|
|     2017-12-21 0:00|            83|
|     2018-03-29 0:00|            83|
|     2018-04-07 0:00|            82|
|     2018-03-25 0:00|            79|
|     2018-03-30 0:00|            78|
|     2018-03-22 0:00|            78|
|     2018-04-04 0:00|            76|
|     2018-03-28 0:00|            74|
|     2018-03-15 0:00|            72|
|     2018-03-18 0:00|            72|
|     2018-04-12 0:00|            72|
|     2017-12-22 0:00|            68|
|     2018-03-17 0:00|            68|
|     2017-12-14 0:00|            66|
|     2018-04-05 0:00|            66|
+--------------------+--------------+
only showing top 20 rows



In [89]:
query = """
SELECT 
    review_score, COUNT(review_id)
FROM 
    order_reviews
GROUP BY 
    review_score
ORDER BY
    1 DESC;
"""

spark.sql(query).show()

+------------+----------------+
|review_score|count(review_id)|
+------------+----------------+
|           5|           57328|
|           4|           19142|
|           3|            8179|
|           2|            3151|
|           1|           11424|
+------------+----------------+



In [91]:
spark.sql("SELECT COUNT(review_id) FROM order_reviews").show()

+----------------+
|count(review_id)|
+----------------+
|           99224|
+----------------+



In [96]:
'리뷰에 제목이 있는 글 | ' + str((11568  / 99224) * 100) + '% |'

'리뷰에 제목이 있는 글 | 11.658469725066515% |'

In [98]:
spark.sql("SELECT * FROM order_items").show()

+-----+--------------------+-------------+--------------------+--------------------+-------------------+------+-------------+
|index|            order_id|order_item_id|          product_id|           seller_id|shipping_limit_date| price|freight_value|
+-----+--------------------+-------------+--------------------+--------------------+-------------------+------+-------------+
|    0|00010242fe8c5a6d1...|            1|4244733e06e7ecb49...|48436dade18ac8b2b...|2017-09-19 09:45:35|  58.9|        13.29|
|    1|00018f77f2f0320c5...|            1|e5f2d52b802189ee6...|dd7ddc04e1b6c2c61...|2017-05-03 11:05:13| 239.9|        19.93|
|    2|000229ec398224ef6...|            1|c777355d18b72b67a...|5b51032eddd242adc...|2018-01-18 14:48:30| 199.0|        17.87|
|    3|00024acbcdf0a6daa...|            1|7634da152a4610f15...|9d7a1d34a50524090...|2018-08-15 10:10:18| 12.99|        12.79|
|    4|00042b26cf59d7ce6...|            1|ac6c3623068f30de0...|df560393f3a51e745...|2017-02-13 13:57:51| 199.9|       

In [101]:
query = """
SELECT 
    order_id,
    COUNT(DISTINCT(product_id)) as count_product,
    COUNT(DISTINCT(seller_id)) as count_seller,
    MAX(order_item_id) as count_item
FROM 
    order_items
GROUP BY 
    order_id
ORDER BY 
3 DESC;
"""

spark.sql(query).show()

+--------------------+-------------+------------+----------+
|            order_id|count_product|count_seller|count_item|
+--------------------+-------------+------------+----------+
|cf5c8d9f52807cb2d...|            6|           5|         6|
|1c11d0f4353b31ac3...|            6|           5|         7|
|8c2b13adf3f377c8f...|            4|           4|         4|
|1d23106803c48c391...|            4|           4|         4|
|91be51c856a90d7ef...|            4|           4|         4|
|2b0d2d60eaf705257...|            3|           3|         5|
|14f0429b74bbb8626...|            3|           3|         3|
|62ce4e3989a347792...|            3|           3|         6|
|70b7e94ea46d3e8b5...|            3|           3|         3|
|306b4ec62d908535f...|            3|           3|         3|
|d4ab53c2b21a94c8b...|            4|           3|         4|
|309b9de8813533826...|            3|           3|         5|
|9d817e85739426de1...|            3|           3|         3|
|cbb7694680a105281...|  

In [105]:
query = """
SELECT 
    order_id,
    COUNT(DISTINCT(seller_id)) AS sellers_per_orders,
    COUNT(DISTINCT(product_id)) AS product_per_orders,
    MAX(shipping_limit_date) AS shipping_limit,
    AVG(price) AS avg_price,
    AVG(freight_value) AS avg_freight
    
FROM 
    order_items
GROUP BY 
    order_id;
"""

spark.sql(query).show()

+--------------------+------------------+------------------+-------------------+---------+-----------+
|            order_id|sellers_per_orders|product_per_orders|     shipping_limit|avg_price|avg_freight|
+--------------------+------------------+------------------+-------------------+---------+-----------+
|00018f77f2f0320c5...|                 1|                 1|2017-05-03 11:05:13|    239.9|      19.93|
|00042b26cf59d7ce6...|                 1|                 1|2017-02-13 13:57:51|    199.9|      18.14|
|00054e8431b9d7675...|                 1|                 1|2017-12-14 12:10:31|     19.9|      11.85|
|0005a1a1728c9d785...|                 1|                 1|2018-03-26 18:31:29|   145.95|      11.65|
|0005f50442cb953dc...|                 1|                 1|2018-07-06 14:10:56|    53.99|       11.4|
|00063b381e2406b52...|                 1|                 1|2018-07-31 17:30:39|     45.0|      12.98|
|0006ec9db01a64e59...|                 1|                 1|2018-07-26 17

In [107]:
spark.sql("SELECT * FROM products").show()

+-----+--------------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+
|index|          product_id|product_category_name|product_name_lenght|product_description_lenght|product_photos_qty|product_weight_g|product_length_cm|product_height_cm|product_width_cm|
+-----+--------------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+
|    0|1e9e8ef04dbcff454...|           perfumaria|               40.0|                     287.0|               1.0|           225.0|             16.0|             10.0|            14.0|
|    1|3aa071139cb16b67c...|                artes|               44.0|                     276.0|               1.0|          1000.0|             30.0|             18.0|            20.0|
|    2|96bd76ec8810374ed...|        esporte_lazer|               

In [109]:
query = """
SELECT 
    product_category_name, COUNT(product_id) AS QTD
FROM 
    products
GROUP BY 
    product_category_name
ORDER BY 
    2 DESC;
"""

spark.sql(query).show()

+---------------------+----+
|product_category_name| QTD|
+---------------------+----+
|      cama_mesa_banho|3029|
|        esporte_lazer|2867|
|     moveis_decoracao|2657|
|         beleza_saude|2444|
| utilidades_domest...|2335|
|           automotivo|1900|
| informatica_acess...|1639|
|           brinquedos|1411|
|   relogios_presentes|1329|
|            telefonia|1134|
|                bebes| 919|
|           perfumaria| 868|
| fashion_bolsas_e_...| 849|
|            papelaria| 849|
|           cool_stuff| 789|
|   ferramentas_jardim| 753|
|             pet_shop| 719|
|                 null| 610|
|          eletronicos| 517|
| construcao_ferram...| 400|
+---------------------+----+
only showing top 20 rows



In [112]:
query = """
SELECT 
    AVG(product_name_lenght) AS avg_name_lenght,
    AVG(product_description_lenght) AS avg_description_name,
    AVG(product_photos_qty) AS avg_photos_qty,
    AVG(product_weight_g) AS avg_weight_g,
    AVG(product_length_cm) AS  avg_length_cm,
    AVG(product_height_cm) AS avg_height_cm,
    AVG(product_width_cm) AS avg_width_cm

FROM 
    products;
"""

spark.sql(query).show()

+-----------------+--------------------+------------------+------------------+-----------------+------------------+------------------+
|  avg_name_lenght|avg_description_name|    avg_photos_qty|      avg_weight_g|    avg_length_cm|     avg_height_cm|      avg_width_cm|
+-----------------+--------------------+------------------+------------------+-----------------+------------------+------------------+
|48.47694876472589|   771.4952846232337|2.1889861166939797|2276.4724877841513|30.81507784758263|16.937661234028347|23.196728277034204|
+-----------------+--------------------+------------------+------------------+-----------------+------------------+------------------+



In [115]:
spark.sql("SELECT * FROM product_category_name_translation").show()

+-----+---------------------+-----------------------------+
|index|product_category_name|product_category_name_english|
+-----+---------------------+-----------------------------+
|    0|         beleza_saude|                health_beauty|
|    1| informatica_acess...|         computers_accesso...|
|    2|           automotivo|                         auto|
|    3|      cama_mesa_banho|               bed_bath_table|
|    4|     moveis_decoracao|              furniture_decor|
|    5|        esporte_lazer|               sports_leisure|
|    6|           perfumaria|                    perfumery|
|    7| utilidades_domest...|                   housewares|
|    8|            telefonia|                    telephony|
|    9|   relogios_presentes|                watches_gifts|
|   10|    alimentos_bebidas|                   food_drink|
|   11|                bebes|                         baby|
|   12|            papelaria|                   stationery|
|   13| tablets_impressao...|         ta

In [117]:
spark.sql("SELECT * FROM sellers").show()

+-----+--------------------+----------------------+-----------------+------------+
|index|           seller_id|seller_zip_code_prefix|      seller_city|seller_state|
+-----+--------------------+----------------------+-----------------+------------+
|    0|3442f8959a84dea7e...|                 13023|         campinas|          SP|
|    1|d1b65fc7debc3361e...|                 13844|       mogi guacu|          SP|
|    2|ce3ad9de960102d06...|                 20031|   rio de janeiro|          RJ|
|    3|c0f3eea2e14555b6f...|                  4195|        sao paulo|          SP|
|    4|51a04a8a6bdcb23de...|                 12914|braganca paulista|          SP|
|    5|c240c4061717ac180...|                 20920|   rio de janeiro|          RJ|
|    6|e49c26c3edfa46d22...|                 55325|           brejao|          PE|
|    7|1b938a7ec6ac5061a...|                 16304|        penapolis|          SP|
|    8|768a86e36ad6aae3d...|                  1529|        sao paulo|          SP|
|   

In [196]:
query = """
SELECT
    product_category_name_english,
    Total_Sales,
    dense_rank() over w as Categ_Sales_Ranking,
    sum(Total_Sales) over w as Accumulated_Sales
FROM
    (SELECT
        product_category_name_english,
        round(sum(payment_value), 2) as Total_Sales
    FROM 
        products as p
    INNER JOIN 
        product_category_name_translation as pt 
    ON 
        pt.product_category_name = p.product_category_name
    INNER JOIN 
        order_items as oi 
    ON 
        oi.product_id = p.product_id
    INNER JOIN 
        order_payments as pay 
    ON
        pay.order_id = oi.order_id
    GROUP BY 
        product_category_name_english
    ORDER BY 
        sum(payment_value) desc) as a
    window w as (order by Total_Sales desc);
"""

In [197]:
spark.sql(query).show()

23/03/24 13:36:28 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/24 13:36:28 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/24 13:36:33 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/24 13:36:35 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/24 13:36:37 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/24 13:36:39 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/24 1

+-----------------------------+-----------+-------------------+--------------------+
|product_category_name_english|Total_Sales|Categ_Sales_Ranking|   Accumulated_Sales|
+-----------------------------+-----------+-------------------+--------------------+
|               bed_bath_table| 1712553.67|                  1|          1712553.67|
|                health_beauty| 1657373.12|                  2|          3369926.79|
|         computers_accesso...| 1585330.45|                  3|          4955257.24|
|              furniture_decor| 1430176.39|                  4|          6385433.63|
|                watches_gifts| 1429216.68|                  5|          7814650.31|
|               sports_leisure| 1392127.56|                  6|          9206777.87|
|                   housewares| 1094758.13|                  7|         1.0301536E7|
|                         auto|  852294.33|                  8|       1.115383033E7|
|                 garden_tools|  838280.75|                  9|  

In [208]:
query = """
SELECT 
    * 
FROM
    (SELECT
        c.customer_id,
        FIRST(o.order_id),
        FIRST(c.customer_state),
        FIRST(cat.product_category_name_english),
        sum(payment_value) as Total_Payment_Value
    FROM 
        order_payments as p
    INNER JOIN 
        orders as o 
    ON 
        o.order_id = p.order_id
    INNER JOIN 
        customers as c 
    ON 
        c.customer_id = o.customer_id
    INNER JOIN 
        order_items as oi 
    ON 
        oi.order_id = p.order_id
    INNER JOIN 
        products as p
    ON p.product_id = oi.product_id
    INNER JOIN 
        product_category_name_translation as cat 
    ON cat.product_category_name = p.product_category_name
    GROUP BY 
        c.customer_id) AS a
WHERE Total_Payment_Value > 1000;
"""

In [209]:
spark.sql(query).show()

+--------------------+--------------------+---------------------+------------------------------------+-------------------+
|         customer_id|     first(order_id)|first(customer_state)|first(product_category_name_english)|Total_Payment_Value|
+--------------------+--------------------+---------------------+------------------------------------+-------------------+
|000598caf2ef41174...|9b961b894e797f636...|                   MG|                            pet_shop|            1255.71|
|0049e8442c2a3e4a8...|442693e8e705ad1f9...|                   SP|                           computers|            1250.81|
|00c634fb2feeb4e27...|dd4d3cf66aa23e931...|                   RJ|                       health_beauty|            1318.06|
|00f394e6fc446865a...|a29c25c2c9c3100da...|                   RJ|                       health_beauty|             1449.5|
|01095519ab7592741...|95820ac4fd93f004a...|                   SP|                computers_accesso...| 2002.6799999999998|
|010be179fab3870

In [131]:
query = """
SELECT 
    i.seller_id AS id,
    FIRST(s.seller_city) AS seller_city,
    FIRST(s.seller_state) AS seller_state,
    COUNT(i.order_id) AS order_id,
    SUM(i.price) AS sum_price,
    SUM(i.freight_value) AS f_v,
    AVG(i.price) AS avg_price,
    AVG(i.freight_value) avg_f_v,
    SUM(p.payment_value) sum_p_v,
    AVG(p.payment_value) AS avg_p_v

FROM 
    order_items as i
    INNER JOIN 
        sellers as s
    ON 
        i.seller_id = s.seller_id
    INNER JOIN 
        order_payments as p
    ON 
        p.order_id = i.order_id
GROUP BY 
    i.seller_id
ORDER BY 
    COUNT(i.order_id) DESC;
"""

spark.sql(query).show()

+--------------------+--------------------+------------+--------+------------------+------------------+------------------+------------------+------------------+------------------+
|                  id|         seller_city|seller_state|order_id|         sum_price|               f_v|         avg_price|           avg_f_v|           sum_p_v|           avg_p_v|
+--------------------+--------------------+------------+--------+------------------+------------------+------------------+------------------+------------------+------------------+
|4a3ca9315b744ce9f...|            ibitinga|          SP|    2133|213299.56999999427| 37335.72000000001| 99.99979840599825| 17.50385372714487| 301245.2700000021| 141.2307876230671|
|6560211a19b47992c...|           sao paulo|          SP|    2122|128489.82000000004|29277.639999999785| 60.55128180961359|13.797191328934865|179657.75000000073| 84.66434967012287|
|1f50f920176fa81da...|sao jose do rio p...|          SP|    2008|  111435.809999997| 37274.849999999

In [126]:
query = """

SELECT 
  c.customer_unique_id , 
  COUNT(DISTINCT(c.customer_id)) ,
  COUNT(DISTINCT(o.order_id)) ,
  COUNT(DISTINCT(i.product_id)),
  MAX(order_purchase_timestamp)
  
FROM 
    customers AS c
    INNER JOIN 
        orders AS o
    ON 
        c.customer_id = o.customer_id
    INNER JOIN 
        order_items AS i
    ON 
        o.order_id = i.order_id
        
GROUP BY 
    c.customer_unique_id
HAVING 
    COUNT(DISTINCT(i.product_id)) > 1
ORDER BY 
    3 DESC;
"""

spark.sql(query).show()

+--------------------+---------------------------+------------------------+--------------------------+-----------------------------+
|  customer_unique_id|count(DISTINCT customer_id)|count(DISTINCT order_id)|count(DISTINCT product_id)|max(order_purchase_timestamp)|
+--------------------+---------------------------+------------------------+--------------------------+-----------------------------+
|8d50f5eadf50201cc...|                         16|                      16|                        15|          2018-08-20 19:14:26|
|3e43e6105506432c9...|                          9|                       9|                        13|          2018-02-27 18:36:39|
|1b6c7548a2a1f9037...|                          7|                       7|                         9|          2018-02-14 13:22:12|
|6469f99c1f9dfae77...|                          7|                       7|                         5|          2018-06-28 00:43:34|
|ca77025e7201e3b30...|                          7|                   

In [133]:
query = """
SELECT 
    *
FROM 
    customers
WHERE 
    customer_unique_id == '8d50f5eadf50201ccdcedfb9e2ac8455'
"""

spark.sql(query).show()

+-----+--------------------+--------------------+------------------------+-------------+--------------+
|index|         customer_id|  customer_unique_id|customer_zip_code_prefix|customer_city|customer_state|
+-----+--------------------+--------------------+------------------------+-------------+--------------+
|14186|1bd3585471932167a...|8d50f5eadf50201cc...|                    4045|    sao paulo|            SP|
|15321|a8fabc805e9a10a3c...|8d50f5eadf50201cc...|                    4045|    sao paulo|            SP|
|16654|897b7f72042714efa...|8d50f5eadf50201cc...|                    4045|    sao paulo|            SP|
|36122|b2b13de0770e06de5...|8d50f5eadf50201cc...|                    4045|    sao paulo|            SP|
|38073|42dbc1ad9d560637c...|8d50f5eadf50201cc...|                    4045|    sao paulo|            SP|
|40141|dfb941d6f7b02f57a...|8d50f5eadf50201cc...|                    4045|    sao paulo|            SP|
|48614|65f9db9dd07a4e79b...|8d50f5eadf50201cc...|               

In [194]:
query = """
WITH Location AS (
SELECT 
    o.order_purchase_timestamp,
    i.order_id,
    s.seller_city,
    s.seller_state,
    c.customer_city,
    c.customer_state,

    CASE WHEN s.seller_city = c.customer_city AND s.seller_state = c.customer_state THEN 1 ELSE 0 END same_city,
    CASE WHEN s.seller_state = c.customer_state THEN 1 ELSE 0 END same_state

FROM 
    orders AS o
    INNER JOIN 
        order_items AS i
    ON 
      o.order_id = i.order_id
    LEFT JOIN 
        customers AS c
    ON 
        o.customer_id = c.customer_id
    LEFT JOIN 
        sellers AS s
    ON 
        i.seller_id = s.seller_id
    WHERE CAST(YEAR(o.order_purchase_timestamp) AS int) >= 2018
ORDER BY 
    o.order_purchase_timestamp DESC, i.order_id ASC, i.order_item_id ASC
)

SELECT 

  o.order_id,
  FIRST(o.order_status) AS status,
  FIRST(o.order_purchase_timestamp) AS purchase,
  FIRST(o.order_approved_at) AS approved,
  FIRST(o.order_delivered_carrier_date) AS delivered_carrier,
  FIRST(o.order_delivered_customer_date) AS delivered_customer,
  FIRST(o.order_estimated_delivery_date) AS delivery_estimated,

  CASE WHEN MAX(p.payment_sequential) = 1 THEN 0 ELSE 1 END AS has_sequential,
  CASE WHEN MAX(p.payment_installments) = 1 THEN 0 ELSE 1 END AS has_installments,
  MAX(p.payment_value) as last_payment,
  
  MAX(r.review_score) as score,

  COUNT(DISTINCT(i.seller_id)) as sellers_per_order,
  COUNT(DISTINCT(i.product_id)) as product_per_order,
  MAX(i.shipping_limit_date) as shipping_limit,
  AVG(i.price) AS avg_price,
  AVG(i.freight_value) AS avg_freight,

  AVG(pd.product_name_lenght) AS avg_name_lenght,
  AVG(pd.product_description_lenght) AS avg_description_name,
  AVG(pd.product_photos_qty) AS avg_photos_qty,
  AVG(pd.product_weight_g) AS avg_weight_g,
  AVG(pd.product_length_cm) AS  avg_length_cm,
  AVG(pd.product_height_cm) AS avg_height_cm,
  AVG(pd.product_width_cm) AS avg_width_cm,

  AVG(same_city) AS avg_same_city,
  AVG(same_state) AS avg_same_state
  
FROM 
    orders AS o
    LEFT JOIN 
        order_payments AS p
    ON 
        p.order_id = o.order_id
    LEFT JOIN 
        order_reviews AS r
    ON 
        r.order_id = o.order_id
    LEFT JOIN 
        order_items AS i
    ON 
        i.order_id = o.order_id
    LEFT JOIN 
        products AS pd
    ON 
        i.product_id = pd.product_id
    LEFT JOIN 
        Location AS l
    ON o.order_id = l.order_id

WHERE CAST(YEAR(o.order_purchase_timestamp) AS int) >= 2018
GROUP BY 
    o.order_id
-- ORDER BY o.order_purchase_timestamp DESC;
"""

In [195]:
spark.sql(query).show()

23/03/24 13:30:17 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------+---------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------+----------------+------------+-----+-----------------+-----------------+-------------------+---------+-----------+---------------+--------------------+--------------+------------+-------------+-------------+------------+-------------+--------------+
|            order_id|   status|           purchase|           approved|  delivered_carrier| delivered_customer| delivery_estimated|has_sequential|has_installments|last_payment|score|sellers_per_order|product_per_order|     shipping_limit|avg_price|avg_freight|avg_name_lenght|avg_description_name|avg_photos_qty|avg_weight_g|avg_length_cm|avg_height_cm|avg_width_cm|avg_same_city|avg_same_state|
+--------------------+---------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------+----------------+------------+-----+-----------------+-----